In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.utils.np_utils import to_categorical
from keras import models, layers, optimizers
from keras.layers import Dense, Flatten, Conv2D
from keras import Model
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('df.csv')
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [3]:
df = df.drop(columns=[
    'Late_delivery_risk'
], axis=1)

In [4]:
df_obj = df.select_dtypes(include='object')
df_obj.drop(columns=['Delivery Status'], axis=1, inplace=True)

In [5]:
#one hot encoder
def onehotencoder(col):
    
    ohe = OneHotEncoder()
    ohe_array = ohe.fit_transform((pd.DataFrame(col))).toarray()

    feature_labels = np.array(ohe.categories_).ravel()


    df_obj = pd.DataFrame(ohe_array, columns=feature_labels)
    
    return df_obj

In [6]:
df_ohe = pd.DataFrame()
for c in df_obj.columns:
    one_hot = onehotencoder(df_obj[c])
    df_ohe = pd.concat([df_ohe, one_hot], axis=1) 

In [7]:
df_cont_var = df.select_dtypes(exclude='object')

In [8]:
df_ohe = pd.concat([df_cont_var, df_ohe], axis=1)

In [9]:
df_ohe = pd.concat([df_ohe, df['Delivery Status'] ], axis=1)

In [10]:
df=df_ohe

In [11]:
df.replace(to_replace=['Shipping on time'], value=0, inplace=True)
df.replace(to_replace=['Advance shipping'], value=1, inplace=True)
df.replace(to_replace=['Late delivery'], value=2, inplace=True)

In [12]:
y=df['Delivery Status']
y= to_categorical(y)

In [13]:
# x variable
X = df.drop(columns=['Delivery Status'], axis=1)

In [14]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X[:10000], y[:10000], test_size=.3)

In [15]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=.5)

In [16]:
# this is to mix up the data rows
#from tensor slices will split the data by featrue value dict 
# the batch and shuffle
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [17]:
# another way to build model
model = keras.Sequential(
    [
        layers.Dense(20, activation="relu", name="layer1"),
        layers.Dense(10, activation="sigmoid", name="layer2"),
        layers.Dense(3, name="layer3")
    ]
)

In [18]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(
    train_ds,
    epochs=50,
    batch_size=250,
    validation_data=(X_val, y_val)
)

Epoch 1/50
219/219 [==============================] - 1s 2ms/step - loss: 4.2598 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.5773
Epoch 2/50
219/219 [==============================] - 0s 987us/step - loss: 3.0049 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.5773
Epoch 3/50
219/219 [==============================] - 0s 1ms/step - loss: 3.0049 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.5773
Epoch 4/50
219/219 [==============================] - 0s 969us/step - loss: 3.0049 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.5773
Epoch 5/50
219/219 [==============================] - 0s 958us/step - loss: 3.0049 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.5773
Epoch 6/50
219/219 [==============================] - 0s 981us/step - loss: 3.0049 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.5773
Epoch 7/50
219/219 [==============================] - 0s 1ms/step - loss: 3.0049 - accuracy: 0.5649 - val_loss: 3.0409 - val_accuracy: 0.577

In [20]:
model.evaluate(test_ds)

47/47 [==============================] - 0s 1ms/step - loss: 3.0947 - accuracy: 0.5867


[3.0946741104125977, 0.5866666436195374]

In [21]:
#changing code for git